<a href="https://colab.research.google.com/github/BarnabasLim/Tensorflow/blob/main/TimeSeries_Prediction_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#__Time Series Prediction__


*   Univariate Time Series (Price Vs Time)
  - Time Series that have a single value at each time step



*   Multivariate Time Series (Birth rate and Death Rate Vs time)
  - Time series that have multiple values at each time step.
  - Helps in understanding impact of related Data.	

	
##__Machine learning applied to time series.__
	

*   Forecast/ Prediction
*   Imputation
*   Anomaly Detection
*   Pattern Recognition
	
##__Building blocks / Patterns in time series__
*  __Stationary time series__ are time series that has behavior that does not change over time. If time series is stationary, the more data you have the better.
$$TimeSeries=Trend+Seasonality+Autocorrelation+Noise$$
<table>
<tr>
<th>Patterns</th>
<th>Descriptions</th>
</tr>
<tr>
<td>Trends </td>
<td>Specific general direction of overall time series</td>
</tr>
<tr>
<td>Seasonality</td>
<td> Repeating patterns at regular intervals</td>
</tr>
<tr>
<td>Autocorrelation</td>
<td>- Time Series like this are often described as having memory as steps are dependent on previous ones<br>
    - Correlates with a delayed copy of itself.<br>
    - (e.g., past data at timestamp t-1 helps to predict data at t
  $$v(t)=0.99v(t-1)$$</td>
</tr>
<tr>
<td>Noise</td>
<td>Non-Predictable Random Values</td>
</tr>
<tr>
<td>Innovations</td>
<td>Unpredictable Spikes</td>
</tr>
</table>


 
*  __Non-Stationary time series__ are series such as stock price that may break trend due to a big financial crisis or a big scandal or a disruptive technological breakthrough. Non-Stationary time series would mean that the optimal time window for training would vary.

#Generating Sequences using Python

